In [20]:
import os
import shutil
import pandas as pd
from geopy.distance import geodesic
import pandas as pd
import folium


In [6]:
# get common data files from both years and copy them to the destination folders

def get_common_csv_files(folder1, folder2):
    files_folder1 = set(file for file in os.listdir(folder1) if file.endswith('.csv'))
    files_folder2 = set(file for file in os.listdir(folder2) if file.endswith('.csv'))
    
    common_files = files_folder1.intersection(files_folder2)
    return common_files

def copy_files(source_folder, destination_folder, files_to_copy):
    for file in files_to_copy:
        source_path = os.path.join(source_folder, file)
        destination_path = os.path.join(destination_folder, file)
        shutil.copy2(source_path, destination_path)

# Replace these paths with your actual paths
folder1_path = 'E:/xie/Sensor Files/4. impute/full MICE imputated/2018'
folder2_path = 'E:/xie/Sensor Files/4. impute/full MICE imputated/2019'
destination_folder1_path = 'E:/xie/Sensor Files/5. dataset/full MICE imputed datasets/2018'
destination_folder2_path = 'E:/xie/Sensor Files/5. dataset/full MICE imputed datasets/2019'

common_files = get_common_csv_files(folder1_path, folder2_path)

# Copy common files to destination folders
copy_files(folder1_path, destination_folder1_path, common_files)
copy_files(folder2_path, destination_folder2_path, common_files)

print("Common CSV files copied successfully.")


Common CSV files copied successfully.


In [142]:
# get number of sensor present in one region
# visualize sensor locations on a map for sanity check

def filter_detectors(data, location, ignore):
    if ignore is None:
        filtered_detectors = data[data['Location'].str.contains(location)]
    else:
        filtered_detectors = data[data['Location'].str.contains(location) & ~data['Location'].str.contains(ignore)]
    return filtered_detectors

def plot_map(data):
    # Extract latitude, longitude, detector ID, and location for I-15 detectors
    latitude = data["Latitude"]
    longitude = data["Longitude"]
    detector_id = data["DetectorID"]
    location = data["Location"]

    # Create a map centered at the mean latitude and longitude, with zoom level 10
    map = folium.Map(location=[latitude.mean(), longitude.mean()], zoom_start=10)

    # Add markers for each detector location
    for lat, lon, det_id, loc in zip(latitude, longitude, detector_id, location):
        tooltip_text = f"Detector ID: {det_id}<br>Location: {loc}<br>Latitude: {lat}<br>Longitude: {lon}"
        folium.CircleMarker(location=[lat, lon], radius=5, color='blue', fill=True, fill_color='blue',
                            tooltip=tooltip_text).add_to(map)

    # Display the map
    display(map)

def main(csv_file, location, ignore):
    data = pd.read_csv(csv_file, delimiter=',')
    filtered_data = filter_detectors(data, location, ignore)
    num_sensors = len(filtered_data)
    print(f"Number of sensors found in {location}: {num_sensors}")
    plot_map(filtered_data)


if __name__ == "__main__":
    csv_file = 'E:/xie/common_sensor_data_ignr1.csv'  # Provide the path to your CSV file containing detector information
    #csv_file='E:/xie/Chloe - Reduced Detector Data - all_combined.csv'
    location = '215 SB'  # Location criteria
    #ignore= None
    ignore = 'Ramp|OnRamp|OffRamp|Access'  # Patterns to ignore
    main(csv_file, location, ignore)


Number of sensors found in 215 SB: 17


In [143]:
# Algorithm successfully operates!
# This code filters detectors based on location criteria and ignore patterns, then concatenates speed data
# from sensors starting from a specified sensor, iterating through the closest sensors.
# It utilizes geodesic distance calculation and saves concatenated speed data to a CSV file.
# Also creates an edge list for the sensors.

import pandas as pd
import os
from geopy.distance import geodesic

def filter_detectors(data, location, ignore):
    if ignore is None:
        filtered_detectors = data[data['Location'].str.contains(location)]
    else:
        filtered_detectors = data[data['Location'].str.contains(location) & ~data['Location'].str.contains(ignore)]
    return filtered_detectors

def tsp(sensor_data, start_sensor_id, train_directory, val_test_directory, output_directory, impute_type, location):
    visited = set()
    visited.add(start_sensor_id)

    # Count the number of sensors found
    num_sensors = 1

    # Create an empty DataFrame to store speeds for training
    train_speed_df = pd.DataFrame()
    # Create an empty DataFrame to store speeds for validation/testing
    val_test_speed_df = pd.DataFrame()

    while len(visited) <= len(sensor_data):
        # Find the current sensor
        current_sensor = sensor_data[sensor_data['DetectorID'] == start_sensor_id].iloc[0]

        # Print the current sensor
        print(f"Current Detector ID: {start_sensor_id}")

        # Load speed data for the current sensor from both directories
        file_name = start_sensor_id.replace('_', '.')

        # Determine file extension based on impute_type
        if impute_type == 'mice':
            extension = '_impute.csv'
        else:
            extension = '_filled.csv'

        train_file_path = os.path.join(train_directory, f"{file_name}{extension}")
        val_test_file_path = os.path.join(val_test_directory, f"{file_name}{extension}")
        
        train_speed_data = pd.read_csv(train_file_path)
        val_test_speed_data = pd.read_csv(val_test_file_path)

        # Concatenate speed column for training and validation/testing
        train_speed_df = pd.concat([train_speed_df, train_speed_data['Speed']], axis=1)
        val_test_speed_df = pd.concat([val_test_speed_df, val_test_speed_data['Speed']], axis=1)

        # Find the closest sensor to the current sensor
        min_distance = float('inf')
        closest_sensor_id = None
        current_lat = current_sensor['Latitude']
        current_lon = current_sensor['Longitude']
        
        for index, sensor in sensor_data.iterrows():
            sensor_id = sensor['DetectorID']
            lat = sensor['Latitude']
            lon = sensor['Longitude']

            if sensor_id not in visited:
                distance = geodesic((current_lat, current_lon), (lat, lon)).kilometers
                if distance < min_distance:
                    min_distance = distance
                    closest_sensor_id = sensor_id

        # Update the start_sensor_id for the next iteration
        start_sensor_id = closest_sensor_id
        visited.add(start_sensor_id)

        # Increment the number of sensors found
        num_sensors += 1

    print(f"Total number of sensors found: {num_sensors-1}")

    # Check if the output directory exists, otherwise create it
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Save the concatenated speed data to CSV files with location included in the name
    train_output_file = os.path.join(output_directory, f"train_{impute_type}_{location}.csv")
    val_test_output_file = os.path.join(output_directory, f"val_test_{impute_type}_{location}.csv")
    
    train_speed_df.to_csv(train_output_file, index=False, header=False)
    val_test_speed_df.to_csv(val_test_output_file, index=False, header=False)
    
    print(f"Training data saved to {train_output_file}")
    print(f"Validation/Testing data saved to {val_test_output_file}")

def create_edge_list(sensor_data):
    num_sensors = len(sensor_data)
    edge_list = []

    for i in range(1, num_sensors + 1):
        for j in range(1, num_sensors + 1):
            if i == j:
                continue
            if abs(i - j) == 1:
                weight = 1  # Nodes i and j are adjacent, so weight is 1
            else:
                weight = 0  # Nodes i and j are not adjacent, so weight is 0
            edge_list.append([i, j, weight])

    return edge_list

def save_edge_list(edge_list, filename):
    edge_df = pd.DataFrame(edge_list)
    edge_df.to_csv(filename, index=False, sep='\t', header=False)

if __name__ == "__main__":

    # parameters:
    csv_file = 'E:/xie/common_sensor_data_ignr1.csv'  # Provide the path to your CSV file containing detector information
    location = '215 NB'  # Location criteria
    ignore = 'Ramp|OnRamp|OffRamp|Access'  # Patterns to ignore
    start_sensor_id = '591_2_466' # Start sensor for concatenation - first sensor to start building the sequence

    ###########

    # Define directories for each impute type
    impute_types = ['mice', '0', '-1']
    train_directories = [
        'E:/xie/Sensor Files/4. impute/full mice imputed Common Files/2018',
        'E:/xie/Sensor Files/4. impute/full 0 imputed Common Files/2018',
        'E:/xie/Sensor Files/4. impute/full -1 imputed Common Files/2018'
    ]
    val_test_directories = [
        'E:/xie/Sensor Files/4. impute/full mice imputed Common Files/2019',
        'E:/xie/Sensor Files/4. impute/full 0 imputed Common Files/2019',
        'E:/xie/Sensor Files/4. impute/full -1 imputed Common Files/2019'
    ]
    output_directories = [
        f'E:/xie/Sensor Files/5. dataset/{location}/full mice',
        f'E:/xie/Sensor Files/5. dataset/{location}/full 0',
        f'E:/xie/Sensor Files/5. dataset/{location}/full -1'
    ]

    data = pd.read_csv(csv_file, delimiter=',')
    filtered_data = filter_detectors(data, location, ignore)
    
    # Call the tsp function for each impute type
    for impute_type, train_dir, val_test_dir, out_dir in zip(impute_types, train_directories, val_test_directories, output_directories):
        tsp(filtered_data, start_sensor_id, train_dir, val_test_dir, out_dir, impute_type, location)

    # Create the edge list
    edge_list = create_edge_list(filtered_data)
    
    # Formulate the edge list file name
    edge_list_filename = f'{location.replace(" ", "_")}_edge_list.txt'
    
    # Specify the location to save the edge list file
    save_location = f'E:/xie/Sensor Files/5. dataset/{location}'
    
    # Save the edge list as a text file
    save_edge_list(edge_list, os.path.join(save_location, edge_list_filename))

Current Detector ID: 591_2_466
Current Detector ID: 592_1_468
Current Detector ID: 592_2_470
Current Detector ID: 593_1_472
Current Detector ID: 593_2_474
Current Detector ID: 594_1_476
Current Detector ID: 595_1_481
Current Detector ID: 595_2_482
Current Detector ID: 596_1_485
Current Detector ID: 596_2_487
Current Detector ID: 597_1_488
Current Detector ID: 597_2_490
Current Detector ID: 598_1_492
Current Detector ID: 599_1_495
Current Detector ID: 599_2_497
Current Detector ID: 600_1_500
Current Detector ID: 600_2_502
Current Detector ID: 601_1_503
Total number of sensors found: 18
Training data saved to E:/xie/Sensor Files/5. dataset/215 NB/full mice\train_mice_215 NB.csv
Validation/Testing data saved to E:/xie/Sensor Files/5. dataset/215 NB/full mice\val_test_mice_215 NB.csv
Current Detector ID: 591_2_466
Current Detector ID: 592_1_468
Current Detector ID: 592_2_470
Current Detector ID: 593_1_472
Current Detector ID: 593_2_474
Current Detector ID: 594_1_476
Current Detector ID: 59